### Regressão de dados em painel 

Pacotes

In [3]:
import pandas as pd
import numpy as np
from itertools import product

# Modelo
import  statsmodels.api as sm 
from linearmodels.panel import PanelOLS

In [4]:
panel_data = pd.read_parquet('../Dados/pnad_covid/pnad_covid_tratada.parquet')

In [5]:
panel_data.head()

,Ano,V1013,V1012,UPA,Estrato,V1008,V1016,A003,A001B1,A001B2,...,17,19,20,21,22,23,24,25,27,28
0,2020,5,1,110000016,1110011,1,1,Mulher,2,2,...,False,False,False,False,False,False,True,True,True,True
1,2020,5,1,110000016,1110011,1,1,Mulher,15,2,...,False,False,False,False,False,False,True,True,True,True
2,2020,5,1,110000016,1110011,1,1,Mulher,24,1,...,False,False,False,False,False,False,True,True,True,True
3,2020,5,1,110000016,1110011,1,1,Homem,29,12,...,False,False,False,False,False,False,True,True,True,True
4,2020,5,4,110000016,1110011,10,1,Homem,18,6,...,False,False,False,False,False,False,True,True,True,True


In [6]:
panel_data.columns

Index(['Ano', 'V1013', 'V1012', 'UPA', 'Estrato', 'V1008', 'V1016', 'A003',
       'A001B1', 'A001B2', 'A001B3', 'A004', 'A002', 'A005', 'UF', 'C007',
       'C007B', 'C007D', 'C001', 'C002', 'C003', 'C013', 'F006',
       'domicilio_id', 'grupo_etario', 'Raça', 'Escolaridade', 'formalidade',
       'setor', 'Ocupação', 'data_formatada', 'semana_dt', 'sigla_uf', '12',
       '15', '17', '19', '20', '21', '22', '23', '24', '25', '27', '28'],
      dtype='object')

variáveis de identificação:
- semana_dt = semana de referencia
- domicilio_id = identificação do domicilio

variáveis exógenas:
- A003, sexo
- A004, cor
- A005, escolaridade
- grupo_etario
- formalidade
- setor
- Ocupação
- 12, 15, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, datas (semana de flexibilização em formato dummy)

variável endógena:
C013, trabalho remoto

In [7]:
panel_data = panel_data[['semana_dt', 'domicilio_id', #indetificadores
                         'C013', #endógena
                         'A003', 'A004', 'A005', 'grupo_etario', 'formalidade', 'setor', 'Ocupação', #exógenas
                         '12', '15', '17', '19', '20', '21', '22', '23', '24', '25', '27', '28']]

In [8]:
panel_data.shape

(2508686, 22)

In [9]:
panel_data.dropna(subset='C013', inplace=True)

In [10]:
panel_data.shape

(113093, 22)

### Balanceamento do modelo

In [11]:
# correcting unbalanced data (like ids missing dates along the dataset)
 
ids = list(panel_data['domicilio_id'].unique())
dates = list(panel_data['semana_dt'].unique())

In [12]:
indexes = pd.MultiIndex.from_product(iterables = [ids, dates],names = ['id', 'time'])

In [13]:
balanced_panel = product(ids, dates)

In [14]:
df_aux = pd.DataFrame(data = balanced_panel, columns = ['portfolio_id', 'date'],index= indexes)

In [15]:
#merge the balanced panel with the original dataset

panel_data = pd.merge(df_aux, panel_data, how = 'left', left_on = ['portfolio_id', 'date'], right_on = ['domicilio_id', 'semana_dt'])

In [16]:
#set multiindex to the dataset with portfolio_id and date

panel_data.set_index(['portfolio_id', 'date'], inplace = True)

### Regressão

In [18]:
panel_data['A005']

portfolio_id             date
11000001613Homem1211980  18         Superior completo
                         20                       NaN
                         19                       NaN
                         21                       NaN
                         24                       NaN
                                         ...         
5300298004Homem2271983   39                       NaN
                         42      Fundamental completa
                         44                       NaN
                         45                       NaN
                         43                       NaN
Name: A005, Length: 1023662, dtype: category
Categories (8, object): ['Sem instrução', 'Fundamental incompleto', 'Fundamental completa', 'Médio incompleto', 'Médio completo', 'Superior incompleto', 'Superior completo', 'Pós-graduação, mestrado ou doutorado']

In [19]:
#Variáveis explicativas do modelo
# Adicionando uma constante
exog_variables = panel_data[['A003', 'A004', 'A005', 'grupo_etario', 'formalidade', 'setor', 'Ocupação', '12', '15', '17', '19', '20', '21', '22', '23', '24', '25', '27', '28']]

In [20]:
exog_variables.shape

(1023662, 19)

In [21]:
# estimating the fixed effects model
f_effects_model = PanelOLS( dependent = panel_data.C013, # variável endógena (Y)
exog = exog_variables, # exogenous variables (X matrix)
entity_effects = True, # True -> Fixed Effects
check_rank = False,  # True -> Balanced Panel
drop_absorbed=True)

/home/matheus/miniconda3/envs/monografia/lib/python3.11/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/home/matheus/miniconda3/envs/monografia/lib/python3.11/site-packages/linearmodels/shared/linalg.py:43: RuntimeWarning: invalid value encountered in divide
  normed_var = x.var(0) / np.abs(x).max(0)


In [22]:
f_effects_results = f_effects_model.fit()

/tmp/ipykernel_5349/3205855132.py:1: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

formalidade.Fora do mercado de trabalho, Ocupação.Pessoas afastada devido distanciamento social, Ocupação.Pessoas afastada por outras razões, 22, 23, 24, 25, 27, 28

  f_effects_results = f_effects_model.fit()


In [23]:
f_effects_results.summary

LinAlgError: Singular matrix

---

Efeito do 2º choque - Flexibilização